In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [4]:
img_paths = glob('./images/*.png')

In [6]:
img_paths

['./images/invoice3.png',
 './images/invoice4.png',
 './images/invoice2.png',
 './images/invoice1.png']

In [8]:
all_invoices = pd.DataFrame(columns=['id','text'])

for img_path in  tqdm(img_paths,desc='BusinessCard'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(img_path)
    # extract data and text 
    image = cv2.imread(img_path)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(int)

    useFulData = df.query('conf >= 30')

    # Dataframe
    invoice = pd.DataFrame()
    invoice['text'] = useFulData['text']
    invoice['id'] = filename
    
    # concatenation
    all_invoices = pd.concat((all_invoices,invoice))

BusinessCard: 100%|███████████████████████████████| 4/4 [00:02<00:00,  1.40it/s]


In [9]:
all_invoices

,id,text
4,invoice3.png,1912
5,invoice3.png,Harvest
6,invoice3.png,Lane
12,invoice3.png,
13,invoice3.png,
...,...,...
120,invoice1.png,card
121,invoice1.png,payment:
122,invoice1.png,$685.35
123,invoice1.png,"(Visa,"


In [10]:
all_invoices.to_csv('invoice.csv', index=False)